# Hamoye Week 4 July 2021 Internship Assignment

In this assignment, we will build artificial intelligence algorithms to label satellite image chips with different atmospheric conditions and the different classes of land cover/land use from Kaggle. The algorithms from this project will enable us to understand where, how and why deforestation happens.
The dataset used in this assignment can be downloaded [here](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/data).

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2021-10-19 21:39:59.677492: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-19 21:39:59.677626: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
print(tf.__version__)

2.4.1


In [4]:
# the following is for me to split the dataset.
train = pd.read_csv('/kaggle/input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')
test = pd.read_csv('/kaggle/input/planet-understanding-the-amazon-from-space/sample_submission_v2.csv/sample_submission_v2.csv')

In [5]:
labels = train['tags'].apply(lambda x: x.split(' '))

In [6]:
label_list=[]
for i in labels:
    for j in i:
        if j not in label_list:
            label_list.append(j)

In [7]:
x_train = []
x_test = []
y_train = []

In [8]:
label_map = {l: i for i, l in enumerate(label_list)}

In [9]:
from tqdm import tqdm
import cv2

In [10]:
# reading the images for model training
for f, tags in tqdm(train.values, miniters=1000):
    img = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(img)
    y_train.append(targets)

100%|██████████| 40479/40479 [03:43<00:00, 181.11it/s]


In [11]:
for f, tags in tqdm(test.values, miniters=1000):
    img = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    x_test.append(cv2.resize(img,(64, 64)))

 66%|██████▋   | 40669/61191 [03:38<01:50, 186.27it/s]


error: OpenCV(4.5.3) /tmp/pip-req-build-fvfwe_ss/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [12]:
y_train = np.array(y_train)
x_train = np.array(x_train, np.float32)/255.0
x_test  = np.array(x_test, np.float32)/255.0

In [13]:
print(x_train.shape)
#print(x_test.shape)
print(y_train.shape)

(40479, 64, 64, 3)
(40479, 17)


In [14]:
# Importing needed libraries
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
np.random.seed(1671)

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [15]:
# building the model, 
model = Sequential()
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(17))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0

2021-10-19 21:49:28.975131: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-19 21:49:28.978277: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-19 21:49:28.978313: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-19 21:49:28.978341: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (169bc04d2bd4): /proc/driver/nvidia/version does not exist
2021-10-19 21:49:28.979633: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [16]:
OPTIMIZER = Adam(lr=0.01)  #Adam(lr=0.01), RMSprop .8624, sgd
model.compile(loss='binary_crossentropy', optimizer=OPTIMIZER , metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=1, validation_split=0.25)
# I noticed that SGD and RMSprop have the same val_accuracy 0.8624 when batch_size = 64, epochs=3, verbose=1 and validation_split = 0.25

2021-10-19 21:49:37.672275: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1492205568 exceeds 10% of free system memory.
2021-10-19 21:49:40.472197: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-19 21:49:40.487411: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Epoch 1/3
475/475 [==============================] - 445s 936ms/step - loss: 0.3292 - accuracy: 0.7889 - val_loss: 0.2566 - val_accuracy: 0.8624
Epoch 2/3
475/475 [==============================] - 446s 939ms/step - loss: 0.2602 - accuracy: 0.8571 - val_loss: 0.2558 - val_accuracy: 0.8624
Epoch 3/3
475/475 [==============================] - 445s 938ms/step - loss: 0.2582 - accuracy: 0.8583 - val_loss: 0.2557 - val_accuracy: 0.8624


In [17]:
import gc #to clear the RAM
gc.collect()

1735

In [18]:
y_test=[]
p_test = model.predict(x_test)
y_test.append(p_test)

2021-10-19 22:11:57.841766: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1998962688 exceeds 10% of free system memory.


In [19]:
result = np.array(y_test[0])
result = pd.DataFrame(result, columns = label_list)
result

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
1,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
2,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
3,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
4,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40664,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
40665,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
40666,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142
40667,0.004386,0.77447,0.025313,0.132634,0.014669,0.006071,0.015831,0.007053,0.000336,0.002706,0.013324,0.000191,0.001333,0.000458,0.000635,0.000446,0.000142


In [20]:
from tqdm import tqdm
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))
    


100%|██████████| 40669/40669 [01:03<00:00, 640.34it/s]


In [ ]:
test['tags'][61191] = preds
test.to_csv('sec_submission.csv', index=False)